# Save dict of (fn, n_slices)

In [ ]:
PROJ       = "fastmri"
ds_name    = "knee_singlecoil_val"

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab
from pathlib import Path 

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # mount google drive at /content/drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ALL_PROJ_PATH = Path(ROOT)/"My Drive/fastai_proj/"  # path to all projects 
PROJ_PATH   = ALL_PROJ_PATH/PROJ                  # path to project on PROJ
PROJ_PATH.mkdir(parents=True, exist_ok=True) # make dir in PROJ to house Github

In [ ]:
# imports
%run "$PROJ_PATH/common_constants_env.ipynb" # defines DS2FN - dict of dataset name: (filename, url)

pip install fastai2
git clone fastMRI
pip install ISMRMRD
pip install fasMRI req
sys append /content/fastMRI
imports


# Data path

In [ ]:
DS2DIR = {"knee_singlecoil_val": "singlecoil_val"}

In [ ]:
path_ds = PROJ_PATH/"data"/DS2DIR[ds_name] # path to dataset
path_ds

PosixPath('/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val')

# Get each h5 volume's kspace shape and attr

In [ ]:
def get_h5_files(path, recurse=True, folders=None):
  return get_files(path, extensions=".h5", recurse=recurse, folders=folders)

In [ ]:
%time fnames_all = get_h5_files(path_ds)
len(fnames_all)

CPU times: user 270 µs, sys: 3.93 ms, total: 4.2 ms
Wall time: 21.9 ms


40

In [ ]:
# Compute the size of zero padding in k-space
# We really should have stored this as an attribute in the hdf5 file
# https://github.com/facebookresearch/fastMRI/blob/master/data/mri_data.py
def get_zero_padding_size(data):
  try:
      import ismrmrd
      hdr = ismrmrd.xsd.CreateFromDocument(
          data['ismrmrd_header'][()])
      enc = hdr.encoding[0]
      enc_size = (enc.encodedSpace.matrixSize.x,
                  enc.encodedSpace.matrixSize.y,
                  enc.encodedSpace.matrixSize.z)
      enc_limits_center = enc.encodingLimits.kspace_encoding_step_1.center
      enc_limits_max = enc.encodingLimits.kspace_encoding_step_1.maximum + 1
      padding_left = enc_size[1] // 2 - enc_limits_center
      padding_right = padding_left + enc_limits_max
  except Exception as e:
      padding_left = None
      padding_right = None
      raise e
  return padding_left, padding_right

def get_info(fname):
  data = h5py.File(fname, 'r')
  kspace = data['kspace']
  n_slices = kspace.shape[0]
  padding_left, padding_right = get_zero_padding_size(data)
  return {
          "n_slices": n_slices, 
          "kspace_shape": kspace.shape, 
          "padding_left": padding_left, 
          "padding_right": padding_right
  }

In [ ]:
%time fn2attr = {str(fn): get_info(fn) for fn in fnames_all}

CPU times: user 2.04 s, sys: 11 ms, total: 2.05 s
Wall time: 2.12 s


# Save

In [ ]:
path_out = str(path_ds/"fn2attr.json") # path to output file
path_out

'/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/h5dict.json'

In [ ]:
import json
with open(path_out, 'w', encoding='utf-8') as f:
    json.dump(fn2attr, f, ensure_ascii=False, indent=4)

## Save normalize bins

In [ ]:
import numpy as np
knee_singlecoil_bins = np.array([3.3871757e-07, 1.2047897e-06, 2.1276026e-06, 2.8031234e-06,
       3.3871793e-06, 3.9252932e-06, 4.4390031e-06, 4.9403270e-06,
       5.4373709e-06, 5.9360796e-06, 6.4409433e-06, 6.9559028e-06,
       7.4837581e-06, 8.0274867e-06, 8.5898737e-06, 9.1734846e-06,
       9.7810789e-06, 1.0416418e-05, 1.1083092e-05, 1.1785728e-05,
       1.2529595e-05, 1.3322939e-05, 1.4174205e-05, 1.5095543e-05,
       1.6101530e-05, 1.7212946e-05, 1.8456420e-05, 1.9864552e-05,
       2.1483836e-05, 2.3372895e-05, 2.5595815e-05, 2.8231008e-05,
       3.1365184e-05, 3.5129200e-05, 3.9790604e-05, 4.6021643e-05,
       5.5602330e-05, 7.0693903e-05, 8.7876535e-05, 1.0877507e-04,
       1.5328558e-04, 2.5654701e-04], dtype=np.float32)

# save numpy array as npy file
path_out = str(path_ds/"uni_bins_40.npy") # path to output file
print(path_out)
np.save(path_out, knee_singlecoil_bins)

/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/uni_bins_40.npy


# Reload

In [ ]:
%time reload_h5dict = json.load(Path(path_out))

CPU times: user 1.3 ms, sys: 24 µs, total: 1.32 ms
Wall time: 3.01 ms


In [ ]:
# Pretty Printing JSON string back
print(json.dumps(reload_h5dict, indent = 4, sort_keys=True))

{
    "/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/file1000017.h5": {
        "kspace_shape": [
            34,
            640,
            372
        ],
        "n_slices": 34,
        "padding_left": 19,
        "padding_right": 354
    },
    "/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/file1000026.h5": {
        "kspace_shape": [
            30,
            640,
            372
        ],
        "n_slices": 30,
        "padding_left": 19,
        "padding_right": 354
    },
    "/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/file1000031.h5": {
        "kspace_shape": [
            40,
            640,
            372
        ],
        "n_slices": 40,
        "padding_left": 19,
        "padding_right": 354
    },
    "/content/drive/My Drive/fastai_proj/fastmri/data/singlecoil_val/file1000052.h5": {
        "kspace_shape": [
            38,
            640,
            372
        ],
        "n_slices": 38,
        "paddi